# Reciprocal Rerank Fusion Retriever

In this example, we walk through how you can combine retireval results from multiple queries and multiple indexes.

The retrieved nodes will be reranked according to the `Reciprocal Rerank Fusion` algorithm demonstrated in this [paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf). It provides an effecient method for rerranking retrieval results without excessive computation or reliance on external models.

pip install llama-index-retrievers-bm25

In [1]:
import os 
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv('/home/santhosh/Projects/courses/Pinnacle/.env')

True

In [3]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

## Setup

Download Data

In [4]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/9607a05a923ddf07deee86a56d386b42943ce381/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-03-31 14:16:59--  https://raw.githubusercontent.com/run-llama/llama_index/9607a05a923ddf07deee86a56d386b42943ce381/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  64.1KB/s    in 1.1s    

2024-03-31 14:17:02 (64.1 KB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



In [5]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

Next, we will setup a vector index over the documentation.

In [6]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, chunk_size=256)

## Create a Hybrid Fusion Retriever

In this step, we fuse our index with a BM25 based retriever. This will enable us to capture both semantic relations and keywords in our input queries.

Since both of these retrievers calculate a score, we can use the reciprocal rerank algorithm to re-sort our nodes without using an additional models or excessive computation.

This setup will also query 4 times, once with your original query, and generate 3 more queries.

By default, it uses the following prompt to generate extra queries:

```python
QUERY_GEN_PROMPT = (
    "You are a helpful assistant that generates multiple search queries based on a "
    "single input query. Generate {num_queries} search queries, one on each line, "
    "related to the following input query:\n"
    "Query: {query}\n"
    "Queries:\n"
)
```

First, we create our retrievers. Each will retrieve the top-2 most similar nodes:

In [10]:
from llama_index.retrievers.bm25 import BM25Retriever

In [11]:
vector_retriever = index.as_retriever(similarity_top_k=2)

bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=2
)

Next, we can create our fusion retriever, which well return the top-2 most similar nodes from the 4 returned nodes from the retrievers:

In [12]:
from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    similarity_top_k=2,
    num_queries=4,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    # query_gen_prompt="...",  # we could override the query generation prompt here
)

In [13]:
# apply nested async to run in a notebook
import nest_asyncio

nest_asyncio.apply()

In [14]:
nodes_with_scores = retriever.retrieve(
    "What happened at Interleafe and Viaweb?"
)

Generated queries:
1. History of Interleafe and Viaweb
2. Success stories of Interleafe and Viaweb
3. Impact of Interleafe and Viaweb on the tech industry


In [15]:
for node in nodes_with_scores:
    print(f"Score: {node.score:.2f} - {node.text}...\n-----\n")

Score: 0.05 - Then we'd never have to write anything to run on users' computers. We could generate the sites on the same server we'd serve them from. Users wouldn't need anything more than a browser.

This kind of software, known as a web app, is common now, but at the time it wasn't clear that it was even possible. To find out, we decided to try making a version of our store builder that you could control through the browser. A couple days later, on August 12, we had one that worked. The UI was horrible, but it proved you could build a whole store through the browser, without any client software or typing anything into the command line on the server.

Now we felt like we were really onto something. I had visions of a whole new generation of software working this way. You wouldn't need versions, or ports, or any of that crap. At Interleaf there had been a whole group called Release Engineering that seemed to be at least as big as the group that actually wrote the software. Now you coul

As we can see, both retruned nodes correctly mention Viaweb and Interleaf!

## Use in a Query Engine!

Now, we can plug our retriever into a query engine to synthesize natural language responses.

In [16]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever)

In [17]:
response = query_engine.query("What happened at Interleafe and Viaweb?")

Generated queries:
1. History of Interleafe and Viaweb
2. Success stories of Interleafe and Viaweb
3. Impact of Interleafe and Viaweb on the tech industry


In [19]:
from llama_index.core.response.notebook_utils import display_response

display_response(response)

**`Final Response:`** At Interleaf, there was a whole group called Release Engineering that seemed to be as significant as the group that actually wrote the software. At Viaweb, the founders started a new company that worked via the web, and they received seed funding to launch their software.